In [1]:
import pandas as pd
import numpy as np
import timm
import torchvision.transforms as T

from wildlife_tools.features.deep import DeepFeatures
from wildlife_tools.features.local import SuperPointExtractor, SiftExtractor, AlikedExtractor, DiskExtractor
from wildlife_tools.similarity.cosine import CosineSimilarity
from wildlife_tools.similarity.pairwise.loftr import MatchLOFTR
from wildlife_tools.similarity.pairwise.lightglue import MatchLightGlue
from wildlife_tools.similarity.pairwise.collectors import CollectCounts, CollectCountsRansac, CollectAll

from wildlife_tools.data.dataset import WildlifeDataset
from wildlife_tools.similarity.wildfusion import SimilarityPipeline, WildFusion
from wildlife_tools.similarity.calibration import IsotonicCalibration

metadata = {'metadata':  pd.read_csv('../tests/TestDataset/metadata.csv'), 'root': '../tests/TestDataset'}
transform = T.Compose([T.Resize([224, 224]), T.ToTensor()])
dataset = WildlifeDataset(**metadata, transform=transform)


# WildFusion
- Run wildfusion as mean of
    - LightGlue + SuperPoint
    - LightGlue + ALIKED
    - LightGlue + DISK
    - LightGlue + SIFT
    - LOFTR
    - Deep features of MegaDescriptor-L

Additionally, shortlist is created using MegaDescriptor-L scores. It can be used to significantly speed up the matching pipelines by using only `B` samples from database per query, which are selected based on score in the shortlist.

In [ ]:
pipelines = [

    SimilarityPipeline(
        matcher = MatchLightGlue(features='superpoint'),
        extractor = SuperPointExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='aliked'),
        extractor = AlikedExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='disk'),
        extractor = DiskExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLightGlue(features='sift'),
        extractor = SiftExtractor(),
        transform = T.Compose([
            T.Resize([512, 512]),
            T.ToTensor()
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = MatchLOFTR(pretrained='outdoor'),
        extractor = None,
        transform = T.Compose([
            T.Resize([512, 512]),
            T.Grayscale(),
            T.ToTensor(),
        ]),
        calibration = IsotonicCalibration()
    ),

    SimilarityPipeline(
        matcher = CosineSimilarity(),
        extractor = DeepFeatures(
            model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
        ),
        transform = T.Compose([
            T.Resize(size=(384, 384)),
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]),
        calibration = IsotonicCalibration()
    ),
]


priority_pipeline =  SimilarityPipeline(
    matcher = CosineSimilarity(),
    extractor = DeepFeatures(
        model = timm.create_model('hf-hub:BVRA/wildlife-mega-L-384', num_classes=0, pretrained=True)
    ),
    transform = T.Compose([
        T.Resize(size=(384, 384)),
        T.ToTensor(),
        T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ]),
)

wildfusion = WildFusion(calibrated_pipelines = pipelines, priority_pipeline = priority_pipeline)

In [3]:
wildfusion.fit_calibration(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.69s/it]


In [4]:
wildfusion(dataset, dataset)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.92s/it]


array([[0.99877299, 0.28487387, 0.36920329],
       [0.24108243, 0.99892936, 0.11111111],
       [0.36928767, 0.17491576, 0.99962759]])

In [5]:
wildfusion(dataset, dataset, B=1)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.04s/it]


array([[0.99812824,       -inf,       -inf],
       [      -inf, 0.9984538 ,       -inf],
       [      -inf,       -inf, 0.9996745 ]], dtype=float32)

In [6]:
wildfusion(dataset, dataset, B=2)

100%|█████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.61s/it]


array([[0.9983724 ,       -inf, 0.36974892],
       [0.29660138, 0.99698895,       -inf],
       [0.36926064,       -inf, 0.9996745 ]], dtype=float32)